In [2]:
import seaborn as sns
import joblib

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import

In [75]:
df = pd.read_csv('Dataframe_Final_Comtés_Mois.csv',index_col=0)

In [ ]:
df.head()

In [76]:
df['Etat'] = df.Etat.astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 903456 entries, 0 to 903455
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Année                    903456 non-null  int64  
 1   Etat                     903456 non-null  object 
 2   Mois                     903456 non-null  int64  
 3   TMAX                     903456 non-null  float64
 4   TAVG                     903456 non-null  float64
 5   PCPN                     903456 non-null  float64
 6   Etat_Comté               903456 non-null  object 
 7   Etendue totale des feux  903456 non-null  float64
 8   Nombre de feux           903456 non-null  int64  
 9   Comté                    903456 non-null  int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 75.8+ MB


In [77]:
df['Etat'] = df['Etat'].apply(lambda x: 'State_' + x)
df.head()

,Année,Etat,Mois,TMAX,TAVG,PCPN,Etat_Comté,Etendue totale des feux,Nombre de feux,Comté
0,1992,State_1,1,54.5,44.6,5.62,01-001,0.0,0,1
1,1993,State_1,1,59.3,49.7,6.08,01-001,0.0,0,1
2,1994,State_1,1,52.2,41.7,3.54,01-001,19.6,3,1
3,1995,State_1,1,57.0,46.4,4.95,01-001,2.6,4,1
4,1996,State_1,1,56.0,44.0,7.04,01-001,0.0,0,1


In [19]:
#Creating dataframes for machine learning by state
def getdummies(df):
    state_county = pd.get_dummies(df['Etat_Comté'],prefix='State_County', prefix_sep='_')
    date = pd.get_dummies(df['Année'],prefix='Y', prefix_sep='_')
    df = df.join(date)
    df = df.join(state_county)
    df.drop(columns=['Année','Etat','Etat_Comté','Comté'],inplace=True)
    return df

In [8]:
liste = list(df.Etat.unique())
for state in liste:
    name = str(state)
    data = df[df['Etat'] == state]
    data = getdummies(data)
    data.to_csv(f'{name}.csv')

### Number of fires

In [113]:
#Creating a modelling function
def model(State):

    #Creating dataframe
    df_results = pd.DataFrame()
    nom = []
    Train_MSE = []
    Test_MSE = []
    Train_MAE = []
    Test_MAE = []

    name = str(State)
    df = pd.read_csv(f'{name}.csv',index_col=0)
    

    from sklearn.model_selection import train_test_split
    import sklearn.metrics
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error
    from sklearn.ensemble import RandomForestRegressor
    
    #target and explanatory variables + test and training sets
    features = df.drop(columns=['Etendue totale des feux','Nombre de feux'])
    nbre = df.loc[:,'Nombre de feux']
    etendue = df.loc[:,"Etendue totale des feux"]
    X_train,X_test,y_train,y_test = train_test_split(features,nbre,random_state=42,test_size=0.2)
    
    #model training and metrics
    regressor = RandomForestRegressor()
    regressor.fit(X_train,y_train)
    y_pred_test = regressor.predict(X_test)
    y_pred_test = pd.DataFrame(y_pred_test, columns=['predictions'])
    y_pred_train = regressor.predict(X_train)
    y_pred_train = pd.DataFrame(y_pred_train, columns=['predictions'])
    rfr_mse_test = mean_squared_error(y_test,y_pred_test)
    rfr_mae_test = mean_absolute_error(y_test,y_pred_test)
    rfr_mse_train = mean_squared_error(y_train,y_pred_train)
    rfr_mae_train = mean_absolute_error(y_train,y_pred_train)
    
    #retrieval for metrics
    nom.append(name)
    Train_MSE.append(rfr_mse_train)
    Train_MAE.append(rfr_mae_train)
    Test_MSE.append(rfr_mse_test)
    Test_MAE.append(rfr_mae_test)
    
    #dataset creation by state
    df_results['Etat']=nom
    df_results['Train_MSE']=Train_MSE
    df_results['Train_MAE']=Train_MAE
    df_results['Test_MSE']=Test_MSE
    df_results['Test_MAE']=Test_MAE
    
    #model export for streamlit
    joblib.dump(regressor, f'{name}'+'model.joblib')
    
    return df_results

In [115]:
#Application of the modelling function to States
df_resultats = pd.DataFrame()
for state in liste:
    df_resultats=pd.concat([df_resultats,model(state)], ignore_index=True)

df_resultats

,Etat,Train_MSE,Train_MAE,Test_MSE,Test_MAE
0,State_1,1.321640,0.516892,9.382728,1.374049
1,State_2,8.896527,1.290854,97.771542,3.863681
2,State_3,0.353312,0.256200,2.292581,0.654072
3,State_4,3.075085,0.706353,16.119016,1.745471
4,State_5,0.876268,0.304442,7.688556,0.848861
5,State_6,1.446229,0.566549,9.345303,1.421367
6,State_7,0.015876,0.021592,0.028744,0.037399
7,State_8,1.782384,0.868139,14.700520,2.406119
8,State_9,1.371365,0.722913,9.209851,1.921138
9,State_10,3.186504,0.569639,22.462070,1.524398


In [118]:
df_resultats.max()

Etat           State_9
Train_MSE     8.896527
Train_MAE     1.290854
Test_MSE     97.771542
Test_MAE      3.863681
dtype: object

In [117]:
df_resultats.min()

Etat          State_1
Train_MSE    0.002751
Train_MAE    0.002969
Test_MSE     0.005586
Test_MAE     0.006464
dtype: object

In [92]:
df_resultats.mean()

C:\Users\norac\AppData\Local\Temp\ipykernel_30784\2313732721.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_resultats.mean()


Train_MSE    1.217657
Train_MAE    0.369317
Test_MSE     9.223409
Test_MAE     0.998378
dtype: float64

### Fires' range

In [88]:
def model_etendue(State):

    #Creating dataframe
    df_results = pd.DataFrame()
    nom = []
    Train_MSE = []
    Test_MSE = []
    Train_MAE = []
    Test_MAE = []

    name = str(State)
    df = pd.read_csv(f'{name}.csv',index_col=0)
    from sklearn.model_selection import train_test_split
    import sklearn.metrics
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error
    
    features = df.drop(columns=['Etendue totale des feux','Nombre de feux'])
    nbre = df.loc[:,'Nombre de feux']
    etendue = df.loc[:,"Etendue totale des feux"]
    X_train,X_test,y_train,y_test = train_test_split(features,etendue,random_state=42,test_size=0.2)
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor()
    regressor.fit(X_train,y_train)
    y_pred_test = regressor.predict(X_test)
    y_pred_test = pd.DataFrame(y_pred_test, columns=['predictions'])
    y_pred_train = regressor.predict(X_train)
    y_pred_train = pd.DataFrame(y_pred_train, columns=['predictions'])
    rfr_mse_test = mean_squared_error(y_test,y_pred_test)
    rfr_mae_test = mean_absolute_error(y_test,y_pred_test)
    rfr_mse_train = mean_squared_error(y_train,y_pred_train)
    rfr_mae_train = mean_absolute_error(y_train,y_pred_train)
    
    nom.append(name)
    Train_MSE.append(rfr_mse_train)
    Train_MAE.append(rfr_mae_train)
    Test_MSE.append(rfr_mse_test)
    Test_MAE.append(rfr_mae_test)
    
    df_results['Etat']=nom
    df_results['Train_MSE']=Train_MSE
    df_results['Train_MAE']=Train_MAE
    df_results['Test_MSE']=Test_MSE
    df_results['Test_MAE']=Test_MAE
    return df_results

In [106]:
df_resultats_etendue = pd.DataFrame()
for state in liste:
    df_resultats_etendue=pd.concat([df_resultats_etendue,model_etendue(state)], ignore_index=True)
df_resultats_etendue
    

,Etat,Train_MSE,Train_MAE,Test_MSE,Test_MAE
0,State_1,2.075073e+03,12.588940,8.266556e+03,28.747050
1,State_2,9.255348e+06,360.321490,1.808765e+07,885.812478
2,State_3,2.137043e+03,7.490981,1.462395e+04,19.616660
3,State_4,8.689303e+06,295.760057,2.921567e+07,610.298727
4,State_5,1.725332e+05,43.734933,3.981796e+06,137.536653
5,State_6,2.772198e+02,2.591096,4.525387e+02,5.360830
6,State_7,3.910072e-01,0.081195,8.771616e-01,0.147922
7,State_8,5.953359e+05,86.850907,3.012478e+06,237.722910
8,State_9,4.987492e+03,7.166724,6.245445e+04,21.330298
9,State_10,1.376700e+07,420.561608,1.985407e+08,1250.448908


In [109]:
df_resultats_etendue.mean()

C:\Users\norac\AppData\Local\Temp\ipykernel_30784\2238232354.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_resultats_etendue.mean()


Train_MSE    3.426291e+06
Train_MAE    7.993729e+01
Test_MSE     2.004127e+07
Test_MAE     2.094784e+02
dtype: float64

In [107]:
df_resultats_etendue.to_csv('df_resultats_etendue.csv')
df_resultats.to_csv('df_resultats_nombre.csv')